<div style="text-align: center;">
    <h1> <font style="bold"> Analisis Predictivo Avanzado </font></h1>
    <h2><font style="bold">Trabajo práctico </font></h2>
    <h3><font style="bold">Integrantes:</font></h3>
</div>
<div style="display: flex; justify-content: center;">
    <h4><ul>
        <li>Noguera Abril</li>
        <li>Arbues Lucas</li>
        <li>Alfie Agustin</li>
        </ul>
    </h4>
</div>

In [110]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import shap
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, recall_score
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgbm
from sklearn.model_selection import RandomizedSearchCV

# Funciones: 
from funciones import encoding, balance, Optimization

In [88]:
df = pd.read_pickle('df.pkl')

In [89]:
df.dropna(subset=['conversion'], inplace=True)

In [90]:
df = df.drop(['ROW_ID'], axis=1)

In [91]:
df.shape

(180761, 68)

# Exploración e Implementación del Modelo 

Una vez finalizado el analisis exploratorio de datos (EDA), se procede a la exploracion e implementacion del modelo.

- Train: 80%
- Val: 20%
- Test: de testeo.

In [92]:
train, test = train_test_split(df, test_size=0.20, random_state=42)
train.reset_index(inplace=True,drop=True)
test.reset_index(inplace=True,drop=True)

## Escalamiento:

In [93]:
# Crea una instancia del MinMaxScaler
scaler = MinMaxScaler()

# Variables a Escalar
var_num = train.select_dtypes(include='number')
var_num = var_num.drop(['conversion'], axis =1)
var_num = var_num.columns

# Ajusta y transforma tus datos de entrenamiento
train[var_num] = scaler.fit_transform(train[var_num])

# Transforma tus datos de prueba
test[var_num]= scaler.transform(test[var_num])

## Encoding:

In [94]:
train_encoded = train[['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp','conversion']]
train = train.drop(['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp','conversion'], axis=1)

In [95]:
test_encoded = test[['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp','conversion']]
test = test.drop(['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp','conversion'], axis=1)

In [96]:
trainTE, testTE = encoding(train_encoded[['category_id', 'domain_id', 'item_id','name','conversion']],
                           test_encoded[['category_id', 'domain_id', 'item_id','name','conversion']],
                            ['category_id', 'domain_id', 'item_id', 'name'], 
                            'TE')

In [97]:
train_encoded = train_encoded.drop(['category_id', 'domain_id', 'item_id','name','conversion'], axis=1)
test_encoded = test_encoded.drop(['category_id', 'domain_id', 'item_id','name','conversion'], axis=1)
trainOHE, testOHE = encoding(train_encoded,
                           test_encoded,
                            ['listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 
                            'OHE')

In [98]:
train = train.merge(trainTE, left_index=True, right_index=True, how='inner')
train = train.merge(trainOHE, left_index=True, right_index=True, how='inner')

In [99]:
test = test.merge(testTE, left_index=True, right_index=True, how='inner')
test = test.merge(testOHE, left_index=True, right_index=True, how='inner')

In [100]:
y_train = train['conversion']
X_train = train.drop(['conversion'], axis = 1)
y_test = test['conversion']
X_test = test.drop(['conversion'], axis = 1)

In [138]:
# dfOHE = encoding(df[['category_id', 'domain_id', 'item_id','name','conversion','ROW_ID']], ['category_id', 'domain_id', 'item_id','name'], 'TE').merge(
#     encoding(df.drop(['category_id', 'domain_id', 'item_id','name','conversion'], axis=1), ['listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'OHE'), 
#     left_index=True, right_index=True, how='inner')
# dfOHE['ROW_ID'] = dfOHE['ROW_ID_x']
# dfOHE =dfOHE.drop(['ROW_ID_x','ROW_ID_y'], axis = 1)

In [139]:
# dfLE = encoding(df[['category_id', 'domain_id', 'item_id','name','conversion','ROW_ID']], ['category_id', 'domain_id', 'item_id','name'], 'TE').merge(
#     encoding(df.drop(['category_id', 'domain_id', 'item_id','name','conversion'], axis=1), ['listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'LE'), 
#     left_index=True, right_index=True, how='inner')
# dfLE['ROW_ID'] = dfLE['ROW_ID_x']
# dfLE = dfLE.drop(['ROW_ID_x','ROW_ID_y'], axis = 1)

In [140]:
# dfOE = encoding(df[['category_id', 'domain_id', 'item_id','name','conversion','ROW_ID']], ['category_id', 'domain_id', 'item_id','name'], 'TE').merge(
#     encoding(df.drop(['category_id', 'domain_id', 'item_id','name','conversion'], axis=1), ['listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'OE'), 
#     left_index=True, right_index=True, how='inner')
# dfOE['ROW_ID'] = dfOE['ROW_ID_x']
# dfOE = dfOE.drop(['ROW_ID_x','ROW_ID_y'], axis = 1)

In [141]:
# dfFE = encoding(df, ['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'FE')

In [142]:
# dfTE = encoding(df, ['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'TE')

## Balanceo:

In [101]:
print((y_train == 1).value_counts())

conversion
False    131298
True      13310
Name: count, dtype: int64


In [102]:
# OS, US, LT, NM y SM
X_train, y_train = balance(X_train, y_train, 'LT')

In [103]:
print((y_train == 1).value_counts())

conversion
False    126917
True      13310
Name: count, dtype: int64


## Optimización

### Random Forest:

In [66]:
model = RandomForestClassifier(n_jobs=-1, verbose=1, oob_score=True)

In [67]:
grid = {
         'n_estimators': np.arange(10, 200, 10),  # Número de árboles en el bosque
          'max_features': ['auto', 'sqrt', 'log2'],  # Número de características a considerar en cada split
          'max_depth': np.arange(1, 20),  # Profundidad máxima del árbol
          'min_samples_split': np.arange(2, 20),  # Número mínimo de muestras requeridas para dividir un nodo
          'min_samples_leaf': np.arange(1, 20),  # Número mínimo de muestras requeridas en un nodo hoja
          'bootstrap': [True, False],  # Método para muestrear instancias de entrenamiento (con o sin reemplazo)
          'criterion': ['gini', 'entropy']  # Función para medir la calidad de una división
 }

### Random Search:

In [68]:
optimizer = Optimization(model=model, param_grid=grid, n_iter=10, cv=5, random_state=42)

In [69]:
%%capture
RS = optimizer.random_search(X_train, y_train)

In [70]:
best_params_random = RS.best_params_
print("Best hyperparameters (Random Search):", best_params_random)

Best hyperparameters (Random Search): {'n_estimators': 190, 'min_samples_split': 9, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'max_depth': 16, 'criterion': 'entropy', 'bootstrap': True}


In [71]:
RS.best_score_

0.9154656496311798

In [72]:
y_proba = RS.predict(X_test)
y_pred = (y_proba >= 0.5).astype(int)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    0.0s
[Parallel(n_jobs=10)]: Done 190 out of 190 | elapsed:    0.1s finished


In [188]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.95      0.37      0.53     32719
         1.0       0.12      0.80      0.21      3434

    accuracy                           0.41     36153
   macro avg       0.53      0.59      0.37     36153
weighted avg       0.87      0.41      0.50     36153



In [30]:
# Calculamos Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculamos Recall
recall = recall_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Recall: {recall}")

# Calculamos Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print(f"Specificity: {specificity}")

Accuracy: 0.9081404032860344
Recall: 0.1432731508444962
Specificity: 0.9884165163972004


In [31]:
best_estimator = RS.best_estimator_

In [32]:
explainer = shap.TreeExplainer(best_estimator)
shap_values = explainer.shap_values(X_train)

# Create a summary plot
shap.summary_plot(shap_values, X_train)

### XGBoost

In [34]:
model = XGBClassifier(n_jobs=-1)

In [35]:
grid = {'max_depth': stats.randint(1,20),
              'learning_rate': stats.uniform(0.01, 0.5),
              'subsample': stats.uniform(0.1, 1),
              'colsample_bytree': stats.uniform(0.1, 1),
              'min_child_weight': stats.randint(1,4),
              'n_estimators': stats.randint(50, 1000)
             }

### Random Search:

In [36]:
optimizer = Optimization(model=model, param_grid=grid, n_iter=10, cv=5, random_state=42)

In [37]:
%%capture
RS = optimizer.random_search(X_train, y_train)

In [38]:
best_params_random = RS.best_params_
print("Best hyperparameters (Random Search):", best_params_random)

Best hyperparameters (Random Search): {'colsample_bytree': 0.2559945203362026, 'learning_rate': 0.03904180608409973, 'max_depth': 4, 'min_child_weight': 3, 'n_estimators': 711, 'subsample': 0.15641157902710026}


In [39]:
RS.best_score_

0.7625130125682886

In [40]:
y_proba = RS.predict(X_test)
y_pred = (y_proba >= 0.5).astype(int)

In [154]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.95      0.37      0.53     32719
         1.0       0.12      0.80      0.21      3434

    accuracy                           0.41     36153
   macro avg       0.53      0.59      0.37     36153
weighted avg       0.87      0.41      0.50     36153



In [ ]:
# Calculamos Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

# Calculamos Recall
recall = recall_score(y_test, y_pred, pos_label=1, average='binary')
print(f"Recall: {recall}")

# Calculamos Specificity
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
print(f"Specificity: {specificity}")

Accuracy: 0.9081404032860344
Recall: 0.1432731508444962
Specificity: 0.9884165163972004


In [ ]:
best_estimator = RS.best_estimator_

In [ ]:
explainer = shap.TreeExplainer(best_estimator)
shap_values = explainer.shap_values(X_train)

# Create a summary plot
shap.summary_plot(shap_values, X_train)

## Lightgbm

In [106]:
# Parámetros para LGBM
param_dist = {
    'num_leaves': stats.randint(6, 50), 
    'min_child_samples': stats.randint(100, 500), 
    'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
    'subsample': stats.uniform(loc=0.2, scale=0.8), 
    'colsample_bytree': stats.uniform(loc=0.4, scale=0.6),
    'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
    'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
    'n_estimators': stats.randint(1000, 2000)
}

In [107]:
# Crear modelo LGBM
clf = lgbm.LGBMClassifier(max_depth=-1, random_state=34, metric='None', n_jobs=-1, class_weight='balanced', 
                         scale_pos_weight = sum(y_train == 0) / sum(y_train == 1), boost_from_average=False)

In [108]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=32)

In [111]:
%%capture
# Búsqueda aleatoria
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=10, cv=kf, scoring='roc_auc', n_jobs=-1, random_state=22)

random_search.fit(X_train, y_train)

In [ ]:
random_search.best_params_ 

# {'colsample_bytree': 0.6626083960492597,
#  'min_child_samples': 310,
#  'min_child_weight': 0.1,
#  'n_estimators': 1390,
#  'num_leaves': 21,
#  'reg_alpha': 0,
#  'reg_lambda': 100,
#  'subsample': 0.7259942658487797}

{'colsample_bytree': 0.6626083960492597,
 'min_child_samples': 310,
 'min_child_weight': 0.1,
 'n_estimators': 1390,
 'num_leaves': 21,
 'reg_alpha': 0,
 'reg_lambda': 100,
 'subsample': 0.7259942658487797}

In [ ]:
random_search.best_score_ # 0.8984360454300557

0.8983421747089089